<a href="https://colab.research.google.com/github/NileshCT/Machine_Learning_Models/blob/main/Node_creat_for_kedro_pipeline_On_WeatherAUS_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly.express as px

In [2]:
def get_data():
  """importing dataset
  output=importing dataset and converting into dataframe."""
  df=pd.read_csv("/content/drive/MyDrive/weatherAUS.csv.zip")
  return df

In [141]:
df=get_data()

In [142]:
def data_treatment(df):
  """Drop date column from dataframe.
  Arg=df
  output=Dataframe with no date column."""
  df.drop(["Date"],axis=1,inplace=True)
  return df
data=data_treatment(df)

In [143]:
def target_nan(df):
  """Selecting nan records from target variable.

  Arg=df

  output=Records with all nan values from target variable."""
  a=[(i,index) for i,index in enumerate(df["RainTomorrow"]) if pd.isna(index)]
  t= pd.DataFrame(a).set_index(0)
  return t

In [ ]:
t=target_nan(df)


In [145]:
#Inference data extraction.
def inference(t):
  """Separating inference data

  Arg=t
  
  output=dataframe of inference data."""
  index_nan=t.index.tolist()
  df_inference=df.iloc[index_nan]
  return df_inference
df_inference=inference(t)

In [146]:
#get training data for modelling.
def training_data(df_inference):
  """Separating training data from original dataframe

  Arg=df_inference

  output=New dataframe with training data only."""
  index_nan=t.index.tolist()
  b=[]
  for i in range(df.shape[0]):
    if i not in index_nan:
      b.append(i)
  return b


In [147]:
c=training_data(df_inference)

In [148]:
def get_training_data(b):
  """Displaying training data

  Arg=b

  Output=Seaparate training_data from original dataframe."""
  df_training_data=df.iloc[b]
  return df_training_data

In [ ]:
df_training_data=get_training_data(b)
df_training_data

In [150]:
def treat_missing(df_training_data):
  """treat missing values with ffill and bfill method

  Arg=df_training_data

  output=nan values from training data get fill with the ffill and bfill method."""
  df_treat_training_data=df_training_data.fillna(method="ffill",axis=0).fillna(method="bfill",axis=0)
  return df_treat_training_data
df_treat_training_data=treat_missing(df_training_data)

In [151]:
def training_data_split(df_treat_training_data):
  """Training data spliting ie.separating predictors and response variables.

  Arg=df_treat_training_data

  Output=Target variable get separated from main traing data."""
  X_training=df_treat_training_data.drop(["RainTomorrow"],axis=1)
  y_training=df_treat_training_data["RainTomorrow"]
  return X_training, y_training
X_training, y_training=training_data_split(df_treat_training_data)

In [152]:
from sklearn import preprocessing
L_Encoder=preprocessing.LabelEncoder()
def lebal_encoder(X_training):
  """Lebal encoding on the discrite varibales.

  Arg=X_training

  Output=converting categorical data into numeric variable."""
  X_training["Location"]=L_Encoder.fit_transform(X_training["Location"])
  X_training["WindGustDir"]=L_Encoder.fit_transform(X_training["WindGustDir"])
  X_training["WindDir9am"]=L_Encoder.fit_transform(X_training["WindDir9am"])
  X_training["WindDir3pm"]=L_Encoder.fit_transform(X_training["WindDir3pm"])
  X_training["RainToday"]=L_Encoder.fit_transform(X_training["RainToday"])
  return X_training
X_training=lebal_encoder(X_training)


In [153]:
from sklearn.preprocessing import StandardScaler
scalar=StandardScaler()
def df_transform(X_training):
  """perform process of standardization on X_reaining data to minimizes the skewness from records

  Arg=X_training

  Output=Gets records with some what normal distribution in the records."""
  model=scalar.fit(X_training)
  X_training1=model.transform(X_training)
  return X_training1
X_training1=df_transform(X_training)

In [154]:
from sklearn.model_selection import train_test_split
def modelling(X_training1):
  """Splitting of the x_traing data into X_train,X_test,y_train,y_test data for predicting the output.

  Arg=X_training1

  output=Get splitted data in the form of train and test data."""
  X_train,X_test,y_train,y_test=train_test_split(X_training1,y_training,random_state=0,test_size=0.20)
  return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test=modelling(X_training1)

In [155]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear', random_state=0)
def logReg(X_train,X_test,y_train,y_test):
  """applying the logistic regression algo on the data after split.

  Arg=X_train,X_test,y_train,y_test

  Output=data training and output prediction."""
  logreg.fit(X_train, y_train)
  y_pred_train = logreg.predict(X_train)
  y_pred_test = logreg.predict(X_test)
  return y_pred_train, y_pred_test
y_pred_train,y_pred_test=logReg(X_train,X_test,y_train,y_test)  

In [158]:
from sklearn.metrics import accuracy_score
def r2_score(y_pred_train,y_pred_test):
  """calculate r2_score

  Arg=y_pred_train,y_pred_test

  output=get r2 score to evaluate the model performance on train and test data."""
  r2_score_train=accuracy_score(y_train, y_pred_train)
  r2_score_test=accuracy_score(y_test, y_pred_test)
  return r2_score_train,r2_score_test
r2_score_train=r2_score(y_pred_train,y_pred_test)
r2_score_test=r2_score(y_pred_train,y_pred_test)

In [159]:
r2_score_train
r2_score_test

(0.8413154702252228, 0.8427511515876086)